In [1]:
import sys
sys.path.append('/Users/paolamartire/shocks')

import numpy as np
import matplotlib.pyplot as plt

from grid_maker import make_grid
from shock_zone import shock_direction
from Utilities.operators import calc_div, calc_grad, zero_interpolator, tree_interpolator
import Utilities.prelude


In [ ]:
def shock_surface(x_array, y_array, z_array, Tgrid, idx, x_zone, y_zone):
    for i in range(len(x_zone)):
        ds = shock_direction(x_array, y_array, z_array, Tgrid, idx)

In [3]:
shockzone = np.loadtxt('shockzone_num45.txt')
shockdir = np.loadtxt('shockdir_num45.txt')

x_zone = shockzone[0]
y_zone = shockzone[1]
z_zone = shockzone[2]

x_dir = shockdir[0]
y_dir = shockdir[1]
z_dir = shockdir[2]

ValueError: the number of columns changed from 361 to 1241 at row 4; use `usecols` to select a subset and avoid this error